In [ ]:
# %pip install dataclasses
# %pip install matching-ds-tools

In [ ]:
import json
import datetime
import re
%matplotlib inline
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from queryrunner_client import Client
USER_EMAIL = 'ssadeghi@uber.com'
qclient = Client(user_email=USER_EMAIL)
CONSUMER_NAME = 'intelligentdispatch'

import os
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
from joblib import Parallel, delayed
#num_cores = multiprocessing.cpu_count()  -- 48
n_cores = 4

In [ ]:
from dataclasses import dataclass
import itertools
from typing import *
import numpy as np
import pandas as pd
from queryrunner_client import Client as QRClient
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
import mdstk
from mdstk.data_fetcher.data_fetcher import DataFetcher
from mdstk.data_fetcher.cached_data_fetcher import CachedDataFetcher
import datetime
pd.set_option('display.max_columns', None)

In [ ]:
"""
CASE 1: San Francisco
city_id = 1
vvid = 8
MAX_LAT = 38.19
MIN_LAT = 37.09
MAX_LNG = -121.55
MIN_LNG = -122.60
LAT_CENTER = 37.6
LON_CENTER = -122.2


CASE 2: Detroit
city_id = 50
vvid = 425
MAX_LAT = 42.89
MIN_LAT = 42.01
MAX_LNG = -82.68
MIN_LNG = -83.93
LAT_CENTER = 42.420149389121406
LON_CENTER = -83.15996619595755


Case 3: Philadelphia
city_id = 20
vvid = 663
MAX_LAT = 40.22
MIN_LAT = 39.74
MAX_LNG = -75.46
MIN_LNG = -74.86
LAT_CENTER = 39.95201837418434
LON_CENTER = -75.15727285438611

"""


In [ ]:
### INPUTS

prefix = 'replay'

city_id = 20
vvid = 663
MAX_LAT = 40.22
MIN_LAT = 39.74
MAX_LNG = -74.86
MIN_LNG = -75.46
LAT_CENTER = 39.95201837418434
LON_CENTER = -75.15727285438611
        
datestrs = [  # 1 week
    '2022-11-13',
    '2022-11-14',
    '2022-11-15',
    '2022-11-16',
    '2022-11-17',
    '2022-11-18',
    '2022-11-20',
    '2022-11-21',
    '2022-11-22',
    '2022-11-23',
    '2022-11-24',
    '2022-11-25',
    '2022-11-26',
    '2022-11-27',
    '2022-11-28',
    '2022-11-29',
    '2022-11-30',
    '2022-12-01',
    '2022-12-02',
    '2022-12-03',
    '2022-12-04',
    '2022-12-05',
    '2022-12-06',
    '2022-12-07',
    '2022-12-08',
]


## ML TRAINING

BATCH_SIZE = 256
ITERATIONS = 500000
DISCOUNT = 0.995
VALUE_DEGRADE_LEVEL = 0.85
FRAC_IDLES = 0.35


In [ ]:
np.log(VALUE_DEGRADE_LEVEL) / np.log(DISCOUNT)

### Query trip data

In [ ]:
# data collection

QUERY = """
-- This query has been rewritten by ssadeghi
SELECT
  --Request Information
  ft.datestr,
  ft.city_id,
  ft.uuid as trip_uuid,
  ft.session_id as session_uuid,
  ft.driver_uuid,

  --Time Information,
  ft.request_timestamp_local,
  ft.request_timestamp_utc,
  ft.eta,
  ft.client_upfront_fare_usd as fare,

  --Distances and duration of the request,
  ft.trip_duration_seconds / 60 as duration_min,

  --Rider Pricing information
--  kfk.msg.rider_fare_fs_totals_total as Rider_Price_Total,
--  kfk.msg.rider_fare_fs_totals_total AS Total_Rider_Fare,
--  kfk.msg.context_rider_surge_multiplier AS Rider_Surge_Multiplier,
  
--   Driver origin information
  mez.BASE.accepted_lat as driver_origin_lat,
  mez.BASE.accepted_lng as driver_origin_lng,
  mez.BASE.begintrip_lat as pickup_lat,
  mez.BASE.begintrip_lng as pickup_lng,
  mez.BASE.dropoff_lat as dropoff_lat,
  mez.BASE.dropoff_lng as dropoff_lng

  from rawdata.schemaless_mezzanine_trips_rows mez
--  left join rawdata_user.kafka_hp_fares_intelligence_rides_fare_driveroffer_nodedup kfk ON mez.BASE.uuid = kfk.msg.context_trip_uuid
  join  restricted_dwh.fact_trip ft on mez.BASE.uuid = ft.uuid
--   CROSS JOIN UNNEST(provider_charges) AS t(charges)
where
  TRUE
  and lower(ft.global_product_name)='uberx'
  and lower(ft.status)='completed'
  and ft.city_id = {city_id}
  and date(ft.datestr) = CAST('{datestr}' as date)
  and date(mez.datestr) = CAST('{datestr}' as date)
  and ft.dispatch_type IS NULL  

"""

In [ ]:
# city_id, num_days, datestr

@dataclass
class Query:
    prefix: str
    city_id: int
    datestr: str
    num_days: int
    
    def __post_init__(self):
        self.name = f'{self.prefix}_city{self.city_id}_{self.datestr}'
        self.qry = QUERY.format(city_id=self.city_id, datestr=self.datestr)
        
class MyDataFetcher(DataFetcher):
    def query_many_presto(self, *args, **kwargs):
        return super().query_many_presto(*args, **kwargs)        

In [ ]:

queries = [Query(prefix=prefix, city_id=city_id, datestr=datestr, num_days=1) for datestr in datestrs]

cache_qry_map = {q.name: q.qry for q in queries}

cdf = CachedDataFetcher(
    data_fetcher=MyDataFetcher(
        user_email=USER_EMAIL,
        consumer_name=CONSUMER_NAME,
    ),
    cache_qry_map=cache_qry_map,
    datacenter='phx2',
    datasource='presto-secure',
)

cdf.fetch(bust_cache=False)

In [ ]:
scans = pd.concat(cdf.dfs.values(), axis=0, ignore_index=True) 


In [ ]:
scans.head(10)

In [ ]:
# Calculate new objective function
def clean_df(df):
    df = df[df['fare'].notnull()]
    df = df[df['driver_origin_lng'] < MAX_LNG]
    df = df[df['driver_origin_lng'] > MIN_LNG]
    df = df[df['driver_origin_lat'] < MAX_LAT]
    df = df[df['driver_origin_lat'] > MIN_LAT]
    df['request_timestamp_local'] = pd.to_datetime(df.request_timestamp_local)
    df['weekday'] = df.request_timestamp_local.dt.dayofweek
    df['second_in_day'] = df.request_timestamp_local.dt.hour * 3600 + \
                          df.request_timestamp_local.dt.minute * 60 + \
                          df.request_timestamp_local.dt.second
    df['trip_duration_seconds'] = df.duration_min * 60
    df['total_driver_trip_time'] = df.trip_duration_seconds + df.eta
    df['destination_arrival_time'] = df.total_driver_trip_time + df.second_in_day
    mask = df['destination_arrival_time'] > 24 * 3600
    df['destination_arrival_time'] = df['destination_arrival_time'].mod(24 * 3600)
    df['weekday'][mask] = (df['weekday'][mask] + 1) % 7
#     df['trip_length'][df['trip_length'] <= 100] = 100
#     df = df.drop_duplicates(subset=['job_uuid', 'supply_uuid'])
    df = df.dropna()
    return df


In [ ]:
df = clean_df(scans)

In [ ]:
# df.plot.scatter(x='driver_origin_lat', y='driver_origin_lng', c='Rider_Price_Total')

In [ ]:
df.describe()

### Query supply idling data

In [ ]:
# data collection

QUERY = """
SELECT
sp.earner_uuid as supply_uuid,
sp.datestr as datestr,
sp.city_id as city_id,
sp.duration_ms.open as open_duration,
sp.earner_state as state,
sp.start_timestamp.local as local_time,
sp.location.lat as lat,
sp.location.lng as lng,
sp.flow_type as flow_type
FROM
driver.fact_earner_supply_minute as sp
WHERE
sp.datestr='{datestr}' and
LOWER(sp.flow_type) IN ('uberx', 'p2p') and
sp.city_id={city_id} and
LOWER(sp.earner_state)='open'
LIMIT 1000000
"""


In [ ]:
# city_id, num_days, datestr

prefix = 'supply'

@dataclass
class Query:
    prefix: str
    city_id: int
    datestr: str
    num_days: int
    
    def __post_init__(self):
        self.name = f'{self.prefix}_city{self.city_id}_{self.datestr}'
        self.qry = QUERY.format(city_id=self.city_id, datestr=self.datestr)
        
class MyDataFetcher(DataFetcher):
    def query_many_presto(self, *args, **kwargs):
        return super().query_many_presto(*args, **kwargs)        

In [ ]:
queries = [Query(prefix=prefix, city_id=city_id, datestr=datestr, num_days=1) for datestr in datestrs]

cache_qry_map = {q.name: q.qry for q in queries}

cdf = CachedDataFetcher(
    data_fetcher=MyDataFetcher(
        user_email=USER_EMAIL,
        consumer_name=CONSUMER_NAME,
    ),
    cache_qry_map=cache_qry_map,
    datacenter='phx2',
    datasource='presto-secure',
)

cdf.fetch(bust_cache=False)

In [ ]:
supplies = pd.concat(cdf.dfs.values(), axis=0, ignore_index=True) 
supplies.head(2)


In [ ]:
print(len(supplies))

In [ ]:
# Calculate new objective function
def clean_df(df):
    df = df[df['local_time'].notnull()]
    df = df[df['lng'] < MAX_LNG]
    df = df[df['lng'] > MIN_LNG]
    df = df[df['lat'] < MAX_LAT]
    df = df[df['lat'] > MIN_LAT]    
    df['local_time'] = pd.to_datetime(df.local_time)
    df['weekday'] = df.local_time.dt.dayofweek
    df['second_in_day'] = df.local_time.dt.hour * 3600 + \
                          df.local_time.dt.minute * 60 + \
                          df.local_time.dt.second
    df = df.dropna()
    return df

In [ ]:
supplies = clean_df(supplies)

In [ ]:
fig, ax = plt.subplots()
supplies.plot.scatter(x='lat', y='lng', color = 'red', s = 2, ax=ax)
df.plot.scatter(x='driver_origin_lat', y='driver_origin_lng', color = 'blue', s = 2, ax=ax)

### training offline DQN

In [ ]:
from mini_sim.util import *
from mini_sim.DQN_offlineData import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import plotly.express as px
import plotly.figure_factory as ff
from collections import defaultdict, deque


In [ ]:

idleTripDuration = np.log(VALUE_DEGRADE_LEVEL) / np.log(DISCOUNT)

class training():
    def __init__(self):
        self.device = 'cuda'
        self.valueNetwork = self.neuralNetInit()
        self.targetNetwork = self.neuralNetInit()
#         self.Loss = nn.SmoothL1Loss()
        self.Loss = nn.GaussianNLLLoss()
        self.Optimizer = torch.optim.RMSprop(self.valueNetwork.parameters(), lr = 1e-6)
        self.lossLog = deque([],maxlen = 10000)
        self.batch_size = BATCH_SIZE
        self.df = df
        self.idles = supplies
        self.discount = DISCOUNT
        self.lenDF = len(self.df)
        self.lenSUP = len(self.idles)
        
    def batchMaker(self, n):
        n_trip_samples = int((1 - FRAC_IDLES) * n)
        n_idle_samples = n - n_trip_samples
        sample = self.df.iloc[np.random.choice(self.lenDF, n_trip_samples)]
        sampleIdle = self.idles.iloc[np.random.choice(self.lenSUP, n_idle_samples)]
        batch_v1 = torch.zeros(n, 4).to(self.device)
        batch_v2 = torch.zeros(n, 4).to(self.device)
        batch_eta = torch.zeros(n, 1).to(self.device)
        batch_tripDuration = torch.zeros(n, 1).to(self.device)
        batch_fares = torch.zeros(n, 1).to(self.device)

        ### make samples based on en routes
        batch_v1[:n_trip_samples, 0] = torch.tensor(sample.weekday.to_numpy(), dtype=torch.float)
        batch_v1[:n_trip_samples, 1] = torch.tensor(sample.second_in_day.to_numpy(), dtype=torch.float)
        batch_v1[:n_trip_samples, 2] = torch.tensor(sample.driver_origin_lat.to_numpy(), dtype=torch.float)
        batch_v1[:n_trip_samples, 3] = torch.tensor(sample.driver_origin_lng.to_numpy(), dtype=torch.float)
        batch_eta[:n_trip_samples,0] = torch.tensor(sample.eta.to_numpy(), dtype=torch.float)
        batch_tripDuration[:n_trip_samples,0] = torch.tensor(sample.trip_duration_seconds.to_numpy(), dtype=torch.float)
        batch_fares[:n_trip_samples,0] = torch.tensor(sample.fare.to_numpy(), dtype=torch.float)
        batch_v2[:n_trip_samples, 0] = torch.tensor(sample.weekday.to_numpy(), dtype=torch.float)
        batch_v2[:n_trip_samples, 1] = torch.tensor(sample.destination_arrival_time.to_numpy() , dtype=torch.float)
        batch_v2[:n_trip_samples, 2] = torch.tensor(sample.dropoff_lat.to_numpy(), dtype=torch.float)
        batch_v2[:n_trip_samples, 3] = torch.tensor(sample.dropoff_lng.to_numpy(), dtype=torch.float)
        
        ### make samples based on idles
        batch_v1[n_trip_samples:, 0] = torch.tensor(sampleIdle.weekday.to_numpy(), dtype=torch.float)
        batch_v1[n_trip_samples:, 1] = torch.tensor(sampleIdle.second_in_day.to_numpy(), dtype=torch.float)
        batch_v1[n_trip_samples:, 2] = torch.tensor(sampleIdle.lat.to_numpy(), dtype=torch.float)
        batch_v1[n_trip_samples:, 3] = torch.tensor(sampleIdle.lng.to_numpy(), dtype=torch.float)
        batch_v2[n_trip_samples:] = batch_v1[n_trip_samples:].clone()
        batch_tripDuration[n_trip_samples:,0] = idleTripDuration
        
        return (batch_v1, batch_v2, batch_eta, batch_tripDuration, batch_fares)

    def neuralNetInit(self):
        model = Net(MAX_LAT, MIN_LAT, MAX_LNG, MIN_LNG).to(self.device)
        init_weights(model)
        return model
    
    def targetNetworkGet(self, batch_x):
        self.targetNetwork.eval()
        with torch.no_grad():
            pred, _ = self.targetNetwork(batch_x)
        return pred

    def gradient_clipping(self, net, clip_val=0.1):
        for p in net.parameters():
            p.data.clamp(-clip_val, clip_val)
    
    def updateTargetNetwork(self):
        self.targetNetwork.load_state_dict(self.valueNetwork.state_dict())
        
    def neuralNetworkStep(self):
        self.valueNetwork.train()
        batch = self.batchMaker(self.batch_size)
        batch_v1, batch_v2, batch_eta, batch_tripDuration, batch_fares = batch
        self.Optimizer.zero_grad()
        ## state 1 value is estimated using policy net
        stateValue1, variance = self.valueNetwork(batch_v1.to(self.device))
        ## state 2 value is estimated using target net
        stateValue2 = self.targetNetworkGet(batch_v2.to(self.device))
        ## state 1 new approximate = gamma(idle + eta) * rew + gamma(idle + eta + trip) * state 2
        timeNextState = batch_eta + batch_tripDuration
        discountedNextState = torch.pow(self.discount, timeNextState) * stateValue2
        timeFareCollected = batch_eta
        discountedFare = torch.pow(self.discount, timeFareCollected) * batch_fares
        newState1Estimate = discountedFare + discountedNextState
        loss = self.Loss(stateValue1, newState1Estimate, variance) #+ F.mse_loss(stateValue1, newState1Estimate)
        self.lossLog.append(loss.item())
        loss.backward()
        assert torch.sum(torch.abs(self.valueNetwork.fc1.weight.grad)) > 1e-5
        self.gradient_clipping(self.valueNetwork)
        self.Optimizer.step()




In [ ]:
# initialize model

tr = training()


In [ ]:
for i in range(ITERATIONS):
    tr.neuralNetworkStep()
    if i % 200 == 0:
        tr.updateTargetNetwork()
    if i % 1000 == 0:
        torch.save(tr.targetNetwork.state_dict(), f'checkpoints/LTSV_city{city_id}_vvid{vvid}_gamma{DISCOUNT}_discount{VALUE_DEGRADE_LEVEL}')
        print(f'loss at {i} is {tr.lossLog[-1]}')

In [ ]:
plt.semilogy(tr.lossLog)

In [ ]:
# model evaluation

tr1 = training()
LTSV = Net(MAX_LAT, MIN_LAT, MAX_LNG, MIN_LNG).cuda()
PATH = f'checkpoints/LTSV_city{city_id}_vvid{vvid}_gamma{DISCOUNT}_discount{VALUE_DEGRADE_LEVEL}'
LTSV.load_state_dict(torch.load(PATH))


In [ ]:

def plotMap(tr, n = 5000, weekday = 2, stat = 'mean'):
    LTSV.eval()
    data = np.zeros((24 * n, 4))
    batch_v1, _, _, _, _ = tr.batchMaker(n)
    batch_v1[:, 0] = weekday
    data[:, 1:3] = batch_v1[:, -2:].cpu().repeat(24, 1)
    with torch.no_grad():
        for idx, selectedTime in enumerate(range(0, 24 * 3600, 3600)):
            data[idx * n: (idx + 1) * n, 0] = idx + 1
            batch_v1[:, 1] = selectedTime
#             print(LTSV(batch_v1))
#             sss
            if stat == 'mean':
                out, _ = LTSV(batch_v1)
                out = out.cpu()
            elif stat == 'cov':
                mean, out = LTSV(batch_v1)
                out = torch.sqrt(out) / mean
                out = out.cpu()
            else:
                raise Exception('stat invalid')
            data[idx * n: (idx + 1) * n, -1] = out[:,0]

    hours_df = pd.DataFrame(data = data, columns = ['hour', 'lat', 'lng', 'value'])

    fig = ff.create_hexbin_mapbox(
        data_frame=hours_df, 
        lat="lat",
        lon="lng",
        color='value',
        animation_frame='hour',
        nx_hexagon=100, opacity=0.8,
        min_count=20
    )    

    fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=8, mapbox_center = {"lat": LAT_CENTER, "lon": LON_CENTER},)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 600
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 600
    fig.layout.coloraxis.showscale = True   
    fig.layout.sliders[0].pad.t = 10
    fig.layout.updatemenus[0].pad.t= 10             

    fig.show()
    

In [ ]:
plotMap(tr1, n = 100000, weekday = 6, stat = 'mean')

In [ ]:
plotMap(tr1, n = 100000, weekday = 6, stat = 'cov')

In [ ]:
# (1am, x1, y1) --> (2am, x2, y2) w/ $10
# (4am, x1, y1) --> (4:30am, x3, y3) w/ $8



In [ ]:
sss

In [ ]:
# week, sec, lat, lng
LTSV.eval()
n = 10000
hr = 10
data = np.zeros((n, 4))
data[:,0] = 2
data[:,0] = 24 * hr
data[:,2] = np.random.rand(n) * (MAX_LAT - MIN_LAT) + MIN_LAT
data[:,3] = np.random.rand(n) * (MAX_LNG - MIN_LNG) + MIN_LNG
with torch.no_grad():
    inp = torch.tensor(data).float().cuda()
    mean, out = LTSV(inp)
    out = torch.sqrt(out) / mean
    out = out.cpu()
    data[idx * n: (idx + 1) * n, -1] = out[:,0]

hours_df = pd.DataFrame(data = data, columns = ['hour', 'lat', 'lng', 'value'])

fig = ff.create_hexbin_mapbox(
    data_frame=hours_df, 
    lat="lat",
    lon="lng",
    color='value',
    animation_frame='hour',
    nx_hexagon=100, opacity=0.8,
    min_count=20
)    

fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=8, mapbox_center = {"lat": LAT_CENTER, "lon": LON_CENTER},)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 600
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 600
fig.layout.coloraxis.showscale = True   
fig.layout.sliders[0].pad.t = 10
fig.layout.updatemenus[0].pad.t= 10             

fig.show()